### 승부예측을 위해 데이터를 수집하는 코드
- 3개의 파라미터가 필요 (EPL) 
    1. 경기일자
    2. 팀1
    3. 팀2

In [26]:
import pandas as pd
import soccerdata as sd

class DataCollector: 
    def __init__(self):
        self.fbref = sd.FBref(
            leagues="ENG-Premier League",
            seasons="2425",  # 2024-2025 시즌
            no_cache=True
        )
        self.clubelo = sd.ClubElo()
        self.team_name_map = {
            "Arsenal": "Arsenal",
            "Aston Villa": "Aston Villa",
            "Bournemouth": "Bournemouth",
            "Brentford": "Brentford",
            "Brighton": "Brighton",
            "Chelsea": "Chelsea",
            "Crystal Palace": "Crystal Palace",
            "Everton": "Everton",
            "Fulham": "Fulham",
            "Leeds United": "Leeds United",
            "Liverpool": "Liverpool",
            "Man City": "Man City",
            "Man Utd": "Man Utd",
            "Newcastle": "Newcastle",
            "Nottingham Forest": "Nottingham Forest",
            "Southampton": "Southampton",
            "Spurs": "Spurs",
            "West Ham": "West Ham",
            "Wolves": "Wolves"
        }

    def get_team_elo(self, team_alias, match_date):
        official_name = self.team_name_map.get(team_alias, team_alias)
        match_date = pd.to_datetime(match_date)
        day_before = match_date - pd.Timedelta(days=1)

        try:
            df = self.clubelo.read_by_date(date=day_before).reset_index()
        except Exception as e:
            print(f"Elo 데이터 로딩 실패: {e}")
            return None

        filtered = df[df["team"].str.lower() == official_name.lower()]
        if filtered.empty:
            print(f"Elo 데이터 없음: {official_name} on {day_before.date()}")
            return None
        return filtered.iloc[0]["elo"]

    def get_last_n_matches_goal_diff(self, team_name, n):
        sched = self.fbref.read_schedule().reset_index()
        team_lower = team_name.lower()

        mask = sched["home_team"].str.lower().str.contains(team_lower, na=False) \
            | sched["away_team"].str.lower().str.contains(team_lower, na=False)
        tm = sched.loc[mask].copy()

        tm["date"] = pd.to_datetime(tm["date"])
        tm = tm.sort_values("date")
        tm["score"] = tm["score"].str.replace("–", "-", regex=False)
        tm = tm[tm["score"].str.contains(r"\d+\s*-\s*\d+", na=False)].copy()

        goals = tm["score"].str.extract(r"(\d+)\s*-\s*(\d+)")
        tm["home_goals"] = pd.to_numeric(goals[0], errors="coerce")
        tm["away_goals"] = pd.to_numeric(goals[1], errors="coerce")

        last_n = tm.tail(n)
        if last_n.empty:
            return pd.DataFrame()

        records = []
        for _, row in last_n.iterrows():
            if team_lower in row["home_team"].lower():
                gf, ga = row["home_goals"], row["away_goals"]
                opp = row["away_team"]
            else:
                gf, ga = row["away_goals"], row["home_goals"]
                opp = row["home_team"]

            records.append({
                "date": row["date"].date(),
                "opponent": opp,
                "goals_for": int(gf),
                "goals_against": int(ga),
                "goal_diff": int(gf - ga)
            })

        return pd.DataFrame(records)

    def collect_features(self, match_date, home_team, away_team):
        match_date = pd.to_datetime(match_date)

        def get_stats(team, n):
            df = self.get_last_n_matches_goal_diff(team, 20)
            df = df[df["date"] < match_date.date()]
            df = df.sort_values("date").tail(n)
            gf = df["goals_for"].sum()
            ga = df["goals_against"].sum()
            form = df["goal_diff"].apply(lambda x: 3 if x > 0 else 1 if x == 0 else 0).sum()
            return int(gf), int(ga), int(form)

        gf3h, ga3h, form3h = get_stats(home_team, 3)
        gf5h, ga5h, form5h = get_stats(home_team, 5)
        gf3a, ga3a, form3a = get_stats(away_team, 3)
        gf5a, ga5a, form5a = get_stats(away_team, 5)

        elo_home = self.get_team_elo(home_team, match_date)
        elo_away = self.get_team_elo(away_team, match_date)

        data = {
            "MatchDate": match_date.date(),
            "HomeTeam": home_team,
            "AwayTeam": away_team,
            "GF3Home": gf3h,
            "GA3Home": ga3h,
            "GF5Home": gf5h,
            "GA5Home": ga5h,
            "GF3Away": gf3a,
            "GA3Away": ga3a,
            "GF5Away": gf5a,
            "GA5Away": ga5a,
            "Form3Home": form3h,
            "Form5Home": form5h,
            "Form3Away": form3a,
            "Form5Away": form5a,
            "HomeElo": elo_home,
            "AwayElo": elo_away
        }

        return pd.DataFrame([data])



In [27]:
co = DataCollector()

df = co.collect_features("2025-05-25", "Wolves", "Brentford")
df

[07/03/25 13:34:35] INFO     Saving cached data to C:\Users\ez\soccerdata\data\FBref                 ]8;id=297953;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=17685;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

                    INFO     Saving cached data to C:\Users\ez\soccerdata\data\ClubElo               ]8;id=952523;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=917571;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

[07/03/25 13:34:42] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=400874;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=387663;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:35:07] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=340768;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=287573;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:35:32] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=382163;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=573162;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:35:57] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=379191;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=354706;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

,MatchDate,HomeTeam,AwayTeam,GF3Home,GA3Home,GF5Home,GA5Home,GF3Away,GA3Away,GF5Away,GA5Away,Form3Home,Form5Home,Form3Away,Form5Away,HomeElo,AwayElo
0,2025-05-25,Wolves,Brentford,2,7,6,7,7,6,13,8,0,6,6,12,1732.723511,1809.577148


In [28]:
df.columns

Index(['MatchDate', 'HomeTeam', 'AwayTeam', 'GF3Home', 'GA3Home', 'GF5Home',
       'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away', 'Form3Home',
       'Form5Home', 'Form3Away', 'Form5Away', 'HomeElo', 'AwayElo'],
      dtype='object')

> - 3개의 파라미터는 사용자한테 자연어로 입력 받고 이를 처리하는 함수가 필요
>   - return 값이 경기일자, 팀1, 팀2

In [1]:
import requests
from datetime import datetime
from dateutil.parser import parse
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("X_RAPIDAPI_KEY")

def get_fixture_info(api_key, match_date, team1, team2):
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"

    headers = {
        "X-RapidAPI-Key": api_key,
        "X-RapidAPI-Host": "api-football-v1.p.rapidapi.com"
    }

    params = {
        "date": match_date,
        "league": 39,
        "season": 2024
    }

    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    for fixture in data["response"]:
        home = fixture["teams"]["home"]["name"].lower()
        away = fixture["teams"]["away"]["name"].lower()

        if team1.lower() in [home, away] and team2.lower() in [home, away]:
            return {
                "home_team": fixture["teams"]["home"]["name"],
                "away_team": fixture["teams"]["away"]["name"]
            }

    return None


def extract_match_parameters(user_input: str) -> dict:
    """
    사용자 입력 → match_date, home_team, away_team (API-Football 기반)
    """

    team_kor_to_eng = {
        "아스널": "Arsenal", "아스톤 빌라": "Aston Villa", "본머스": "Bournemouth", "브렌트포드": "Brentford",
        "브라이턴": "Brighton", "첼시": "Chelsea", "크리스탈 팰리스": "Crystal Palace", "에버턴": "Everton",
        "풀럼": "Fulham", "리즈": "Leeds United", "리버풀": "Liverpool", "맨시티": "Man City",
        "맨체스터 시티": "Man City", "맨유": "Man Utd", "맨체스터 유나이티드": "Man Utd",
        "뉴캐슬": "Newcastle", "노팅엄 포레스트": "Nottingham Forest", "사우샘프턴": "Southampton",
        "토트넘": "Spurs", "스퍼스": "Spurs", "웨스트햄": "West Ham", "울브스": "Wolves", "울버햄튼": "Wolves"
    }

    # 날짜 파싱
    try:
        match_date = parse(user_input, fuzzy=True).date()
    except Exception:
        return {"match_date": None, "home_team": None, "away_team": None}

    # 팀 파싱
    found_teams = []
    for kor, eng in team_kor_to_eng.items():
        if kor in user_input and eng not in found_teams:
            found_teams.append(eng)

    if len(found_teams) < 2:
        return {"match_date": str(match_date), "home_team": None, "away_team": None}

    # home/away 정보는 API를 통해 확정
    fixture = get_fixture_info(api_key, str(match_date), found_teams[0], found_teams[1])
    if fixture:
        return {
            "match_date": str(match_date),
            "home_team": fixture["home_team"],
            "away_team": fixture["away_team"]
        }
    else:
        return {
            "match_date": str(match_date),
            "home_team": None,
            "away_team": None
        }


In [2]:
user_input = "5월 25일 울버햄튼과 브렌트포드 경기 알려줘"

params = extract_match_parameters(user_input)
print(params)

{'match_date': '2025-05-25', 'home_team': 'Wolves', 'away_team': 'Brentford'}


### 완성한 코드사용 예제
- 사용자 입력을 받으면 승부예측을 위한 데이터 수집 및 데이터 프레임 생성

In [1]:
from dataParsing.dataCollector import DataCollector
from dataParsing.match_parser import extract_match_parameters

# 1. 사용자 입력
user_input = "5월 25일 울버햄튼과 브렌트포드 경기"

# 2. 한글 입력 → 날짜, 홈팀, 원정팀 추출
params_1 = extract_match_parameters(user_input)

[07/03/25 13:44:59] INFO     No custom team name replacements found. You can configure these in       ]8;id=948974;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=41215;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\ez\soccerdata\config\teamname_replacements.json.                             

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=90766;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=365384;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\ez\soccerdata\config\league_dict.json.                                       

In [2]:
params_1

{'match_date': '2025-05-25', 'home_team': 'Wolves', 'away_team': 'Brentford'}

In [6]:
collector = DataCollector()

# 4. 수집 실행
df_test = collector.collect_features(
    match_date=params_1["match_date"],
    home_team=params_1["home_team"],
    away_team=params_1["away_team"]
)

print("수집된 데이터:")
print(df_test)

[07/03/25 13:45:38] INFO     Saving cached data to C:\Users\ez\soccerdata\data\FBref                 ]8;id=268040;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=374714;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

                    INFO     Saving cached data to C:\Users\ez\soccerdata\data\ClubElo               ]8;id=686097;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=55059;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

[07/03/25 13:45:45] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=888325;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=238869;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:46:10] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=337736;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=596712;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:46:35] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=478573;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=907353;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

[07/03/25 13:47:00] WARNING  c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packag ]8;id=34965;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py\warnings.py]8;;\:]8;id=30682;file://C:\Users\ez\AppData\Local\Programs\Python\Python39\lib\warnings.py#109\109]8;;\
                             es\soccerdata\fbref.py:165: FutureWarning: The behavior of DataFrame                  
                             concatenation with empty or all-NA entries is deprecated. In a future                 
                             version, this will no longer exclude empty or all-NA columns when                     
                             determining the result dtypes. To retain the old behavior, exclude the                
                             relevant entries before the concat operation.                                         
                               pd.concat(dfs)                                                                      
                                                                                                                   

수집된 데이터:
    MatchDate HomeTeam   AwayTeam  GF3Home  GA3Home  GF5Home  GA5Home  \
0  2025-05-25   Wolves  Brentford        2        7        6        7   

   GF3Away  GA3Away  GF5Away  GA5Away  Form3Home  Form5Home  Form3Away  \
0        7        6       13        8          0          6          6   

   Form5Away      HomeElo      AwayElo  
0         12  1732.723511  1809.577148  


In [7]:
print(df_test.columns.tolist())

['MatchDate', 'HomeTeam', 'AwayTeam', 'GF3Home', 'GA3Home', 'GF5Home', 'GA5Home', 'GF3Away', 'GA3Away', 'GF5Away', 'GA5Away', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'HomeElo', 'AwayElo']


In [8]:
df_test

,MatchDate,HomeTeam,AwayTeam,GF3Home,GA3Home,GF5Home,GA5Home,GF3Away,GA3Away,GF5Away,GA5Away,Form3Home,Form5Home,Form3Away,Form5Away,HomeElo,AwayElo
0,2025-05-25,Wolves,Brentford,2,7,6,7,7,6,13,8,0,6,6,12,1732.723511,1809.577148


In [12]:
df_test.to_csv('teamData.csv')

In [1]:
from dataParsing.merge_understat_xg import merge_understat_xg

df_updated = merge_understat_xg(
    csv_path='teamData.csv',
    save_path='teamData_xG.csv',
    seasons=[2024]
)


[07/03/25 14:00:32] INFO     No custom team name replacements found. You can configure these in       ]8;id=698247;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=92471;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#91\91]8;;\
                             C:\Users\ez\soccerdata\config\teamname_replacements.json.                             

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=597845;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=835408;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_config.py#197\197]8;;\
                             C:\Users\ez\soccerdata\config\league_dict.json.                                       

                    INFO     Saving cached data to C:\Users\ez\soccerdata\data\Understat             ]8;id=229170;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=97633;file://c:\Users\ez\Documents\GitHub\AI-Football-Manager\.venv\lib\site-packages\soccerdata\_common.py#263\263]8;;\

✅ 저장 완료: teamData_xG.csv (행 수: 1)


In [2]:
df_updated

,Unnamed: 0,MatchDate,HomeTeam,AwayTeam,GF3Home,GA3Home,GF5Home,GA5Home,GF3Away,GA3Away,...,Form5Away,HomeElo,AwayElo,h_xg,a_xg,xG_diff,xg_margin,xg_ratio,rolling_xg_home_5,rolling_xg_away_5
0,0,2025-05-25,Wolverhampton Wanderers,Brentford,2,7,6,7,7,6,...,12,1732.723511,1809.577148,1.10889,1.97196,-0.86307,0.86307,0.562329,1.10889,1.97196
